In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_name = notebook_path.split("/")[-1]


In [0]:
df = spark.sql('''
    select country_code, platform_version, date_quarter, year, quarter, service_type, group_label
            , UD, UD_ratio, UD_tot
            , duration_hour_median, duration_hour_avg, duration_hour_ratio, duration_hour_sum, duration_hour_std
            , etl_dt
    from sandbox.z_sam351_kim.a0105_yearly_report_7_group_stat_qq
    where country_code IN ('US')
    order by platform_version, date_quarter, service_type, group_label
''')

df.coalesce(1).write\
  .format("com.databricks.spark.csv")\
  .option("header", "true")\
  .save(f's3://s3-lge-he-inbound-aic-dev/HEDS/{notebook_name}/query1_{df.count()}')

df = None

In [0]:
df = spark.sql('''
    select country_code, platform_version, date_ym, year, month, service_type, group_label
            , UD, UD_ratio, UD_tot
            , duration_hour_median, duration_hour_avg, duration_hour_ratio, duration_hour_sum, duration_hour_std
            , etl_dt
    from sandbox.z_sam351_kim.a0105_yearly_report_8_group_stat_mm
    where country_code IN ('US');
''')

df.coalesce(1).write\
  .format("com.databricks.spark.csv")\
  .option("header", "true")\
  .save(f's3://s3-lge-he-inbound-aic-dev/HEDS/{notebook_name}/query2_{df.count()}')

df = None

In [0]:
%sql 

    select country_code, platform_version, date_ym, service_type, group_label, pre_group_label, UD, round(UD_ratio, 6) as UD_ratio, UD_tot, etl_dt
    from sandbox.z_sam351_kim.a0105_yearly_report_9_group_trans_ud_mm
    where country_code IN ('US')
        and date_ym like '2025%'
    order by country_code, platform_version, date_ym, service_type, group_label, pre_group_label;

In [0]:
df = spark.sql('''
    select country_code, platform_version, date_ym, service_type, group_label, pre_group_label, UD, round(UD_ratio, 6) as UD_ratio, UD_tot, etl_dt
    from sandbox.z_sam351_kim.a0105_yearly_report_9_group_trans_ud_mm
    where country_code IN ('US')
        and date_ym like '2025%'
    order by country_code, platform_version, date_ym, service_type, group_label, pre_group_label;
''')

df.coalesce(1).write\
  .format("com.databricks.spark.csv")\
  .option("header", "true")\
  .save(f's3://s3-lge-he-inbound-aic-dev/HEDS/{notebook_name}/query3_{df.count()}')

df = None


In [0]:
df = spark.sql('''
    with t1 as (
    select *
            , sum(UD) over (partition by platform_version, date_ym, service_type, action_label) as UD_tot
            , round(UD / UD_tot, 4) as UD_ratio
    from sandbox.z_sam351_kim.a0105_yearly_report_10_home_entry_stat_mm
    where country_code IN ('US')
    )
    , t2 as (
    select *
            , sum(UD) over (partition by platform_version, date_ym, service_type, action_label) as UD_tot
            , round(UD / UD_tot, 4) as UD_ratio
    from sandbox.z_sam351_kim.a0105_yearly_report_11_home_click_stat_mm
    where country_code IN ('US')
    )
    select 't1_home_usage' as table_nm, country_code, platform_version, date_ym, service_type, group_label, is_nl_matched, action_label
            , UD_ratio, UD, UD_tot, access_median, access_avg, access_ratio, access_sum, access_tot, access_std
            , etl_dt
    from t1
    union all
    select 't2_home_click' as table_nm, country_code, platform_version, date_ym, service_type, group_label, is_nl_matched, action_label
            , UD_ratio, UD, UD_tot, access_median, access_avg, access_ratio, access_sum, access_tot, access_std
            , etl_dt
    from t2
    order by table_nm, country_code, platform_version, date_ym, action_label, service_type, group_label, is_nl_matched;
''')

df.coalesce(1).write\
  .format("com.databricks.spark.csv")\
  .option("header", "true")\
  .save(f's3://s3-lge-he-inbound-aic-dev/HEDS/{notebook_name}/query4_{df.count()}')

df = None

In [0]:
df = spark.sql('''
    select country_code, platform_version, service_type, last_month, last_group_label, first_trans_month, first_pre_group, last_trans_month, last_pre_group
            , sum(UD) as UD
            , etl_dt
    from sandbox.z_sam351_kim.a0105_yearly_report_12_group_trans_month
    where country_code IN ('US')
        and platform_version IN ('webOS24', 'webOS25')
        and service_type NOT IN ('Home')
    group by country_code, platform_version, service_type, last_month, last_group_label, first_trans_month, first_pre_group, last_trans_month, last_pre_group, etl_dt
    order by country_code, platform_version, service_type, last_month, last_group_label, first_trans_month, first_pre_group, last_trans_month, last_pre_group
''')

df.coalesce(1).write\
  .format("com.databricks.spark.csv")\
  .option("header", "true")\
  .save(f's3://s3-lge-he-inbound-aic-dev/HEDS/{notebook_name}/query5_{df.count()}')

df = None




 
 